In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

from control_theory import utils
from control_theory.systems import PendODESystem

In [ ]:
pend = PendODESystem()

## Regulator synthesis

### Let's look at control matrix

In [ ]:
C = np.column_stack([
    pend.b,
    pend.A @ pend.b,
    np.linalg.matrix_power(pend.A, 2) @ pend.b,
    np.linalg.matrix_power(pend.A, 3) @ pend.b
])

C, C.shape

In [ ]:
print(f"Control matrix rank: {np.linalg.matrix_rank(C)}")

In [ ]:
A_eigs = np.linalg.eigvals(pend.A)
print(f"Eigs of the A: {A_eigs}")

### Here we get our control

In [ ]:
theta = (
    -np.array([[0, 0, 0, 1]])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(-1, -2, -7.5, -4)(pend.A)
)

print(f"Our control: {theta}")

In [ ]:
def system(
        t: np.ndarray,
        x: np.ndarray,
        A: np.ndarray,
        b: np.ndarray,
        theta: np.ndarray,
) -> np.ndarray:

    return (A + b @ theta) @ x

In [ ]:
start, stop = 0, 10

time = np.linspace(start, stop, 300)
y_0 = np.array([0, 0.1, 0, 0])

sol = integrate.solve_ivp(system, (start, stop), y_0, dense_output=True, args=(pend.A, pend.b, theta), method="RK45")

In [ ]:
z = sol.sol(time)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time, z[i])
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)

fig.tight_layout()
# fig.savefig('out.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()